<a href="https://colab.research.google.com/github/malcolmrite-dsi/RockVideoClassifier/blob/main/RocksResnetTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.applications import xception
import pandas as pd
import PIL
import matplotlib.pyplot as plt
import os
import shutil

# Training

In [46]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(validation_split=0.2, preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/Module 2 shared folder/samples',
        subset="training",
        seed=3,
        target_size=(64, 64),
        batch_size=64,
        class_mode='categorical')

val_generator = train_datagen.flow_from_directory( '/content/drive/My Drive/Module 2 shared folder/samples', 
        subset="validation",
        seed=3,
        target_size=(64, 64),
        batch_size=64,
        class_mode='categorical')

Found 3132 images belonging to 5 classes.
Found 781 images belonging to 5 classes.


In [12]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/Module 2 shared folder/samples',
        subset="training",
        seed=3,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 3913 images belonging to 5 classes.


In [47]:
resnet = keras.applications.ResNet50(include_top=False, pooling="max", input_shape=(64,64,3))

In [48]:
# mark loaded layers as not trainable
for layer in resnet.layers:
  layer.trainable = False 


In [16]:
data_augmentation = tf.keras.Sequential([
  keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  #keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [49]:
# mark loaded layers as not trainable
#for layer in resnet.layers:
	#layer.trainable = False 
    
flat = Flatten()(resnet.layers[-1].output)
dense = Dense(1024, activation='relu')(flat)
output = Dense(5, activation='softmax')(dense)
model = Model(inputs=resnet.inputs, outputs=output)

In [ ]:
model.summary()

In [50]:
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["categorical_accuracy"])
checkpoint_best = keras.callbacks.ModelCheckpoint("/content/drive/My Drive/model_best.h5", 
            monitor='loss', verbose=0, save_best_only=True, save_weights_only=False, save_freq='epoch')
checkpoint = keras.callbacks.ModelCheckpoint("/content/drive/My Drive/model_last.h5", 
            verbose=0, save_best_only=False, save_weights_only=False, save_freq='epoch')
model.fit(
        train_generator,
        epochs = 3,
        validation_data=val_generator,
        callbacks=[checkpoint_best]
)

Epoch 1/3
49/49 [==============================] - 61s 1s/step - loss: 1.3357 - categorical_accuracy: 0.7995 - val_loss: 0.8146 - val_categorical_accuracy: 0.8707
Epoch 2/3
49/49 [==============================] - 61s 1s/step - loss: 0.3326 - categorical_accuracy: 0.8994 - val_loss: 0.7233 - val_categorical_accuracy: 0.8489
Epoch 3/3
49/49 [==============================] - 61s 1s/step - loss: 0.2178 - categorical_accuracy: 0.9256 - val_loss: 0.5075 - val_categorical_accuracy: 0.8566


In [42]:
model.evaluate(val_generator)

13/13 [==============================] - 36s 3s/step - loss: 0.4126 - categorical_accuracy: 0.8822


[0.4126453995704651, 0.8822023272514343]

In [ ]:
model.fit(
        train_generator,
        initial_epoch=10,
        epochs = 25,
        validation_data=val_generator, callbacks=[checkpoint, checkpoint_best]
)

In [51]:
model.save("/content/drive/My Drive/model_best.h5")